In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv
/kaggle/input/g-research-crypto-forecasting/asset_details.csv
/kaggle/input/g-research-crypto-forecasting/example_test.csv
/kaggle/input/g-research-crypto-forecasting/train.csv
/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/g-research-crypto-forecasting/gresearch_crypto/__init__.py


In [1]:
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import scipy.stats

In [2]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import keras.optimizers 
import math
from sklearn.metrics import mean_squared_error
from keras import backend as K

In [3]:
def calculate_target(data: pd.DataFrame, details: pd.DataFrame, price_column: str):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_ID)
    weights = np.array(list(details.Weight))

    all_timestamps = np.sort(data['timestamp'].unique())
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        targets[asset_names[i]] = (
            price.shift(periods=-16) /
            price.shift(periods=-1)
        ) - 1
    
    targets['m'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['m']

    num = targets.multiply(m.values, axis=0).rolling(3750).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3750).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    targets = targets - (beta * m.values).T
    targets.drop('m', axis=1, inplace=True)
    
    return targets

In [4]:
K.set_floatx('float64')

In [5]:
def create_dataset(dataset,look_back, TargetPrice):
    dataX, dataY1,dataY16 = [], [],[]
    for i in range(TargetPrice+look_back,len(dataset)):
        a = dataset[(i-TargetPrice-look_back):(i-TargetPrice)]
        dataX.append(a)
        dataY1.append(dataset[i-TargetPrice+1])
        dataY16.append(dataset[i])
    return np.array(dataX), np.array(dataY1),np.array(dataY16)

In [24]:
crypto = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
train_df = crypto.drop(columns=['Open', 'High', 'Low','Volume','Count','Target'])


In [25]:
supplement = pd.read_csv("../input/g-research-crypto-forecasting/supplemental_train.csv")
supplement = supplement.drop(columns=['Open', 'High', 'Low','Volume','Count','Target'])

In [9]:
train_df = train_df.append(supplement,ignore_index=True)

In [8]:
details = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')

In [9]:
AssetList1 = ['bnc','btc','bcc','ada','doge','eos','eth','etc','iota','ltc','maker','mxr','xlm','trx']

In [10]:
AssetListT1={}
AssetListT16={}

In [28]:
binance = train_df[train_df["Asset_ID"]==0].reset_index(drop=True)
btc = train_df[train_df["Asset_ID"]==1].reset_index(drop=True) # Asset_ID = 1 for Bitcoin
btc_cash = train_df[train_df["Asset_ID"]==2].reset_index(drop=True)
cardano = train_df[train_df["Asset_ID"]==3].reset_index(drop=True)
dodge = train_df[train_df["Asset_ID"]==4].reset_index(drop=True)  
eos = train_df[train_df["Asset_ID"]==5].reset_index(drop=True)
eth = train_df[train_df["Asset_ID"]==6].reset_index(drop=True)
eth_classic = train_df[train_df["Asset_ID"]==7].reset_index(drop=True)
iota = train_df[train_df["Asset_ID"]==8].reset_index(drop=True)
lite = train_df[train_df["Asset_ID"]==9].reset_index(drop=True)
maker = train_df[train_df["Asset_ID"]==10].reset_index(drop=True)
monero = train_df[train_df["Asset_ID"]==11].reset_index(drop=True)
stellar = train_df[train_df["Asset_ID"]==12].reset_index(drop=True)
tron = train_df[train_df["Asset_ID"]==13].reset_index(drop=True)

In [30]:
listOfDataframe =[binance, btc, btc_cash, cardano, dodge, eos, eth, eth_classic, iota, lite, maker, monero, stellar, tron]

In [76]:
'''
for i in range(len(AssetList1)):
    asset = AssetList1[i]
    model = AssetListT1[name]
    model.save(asset+'1')
    model = AssetListT16[name]
    model.save(asset+'16')
'''

2022-01-31 00:40:35.171098: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [13]:
for i in range(len(listOfDataframe)):
    asset = listOfDataframe[i]
    asset.replace([np.inf, -np.inf], np.nan, inplace=True)
    asset = asset.dropna()
    Close_df = asset[[ 'Close']]
    VWAP_df = asset[[ 'VWAP']]
    Close = Close_df.values
    VWAP = VWAP_df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_Close = scaler.fit_transform(Close)
    scaled_VWAP = scaler.fit_transform(VWAP)
    look_back = 4
    TargetPrice = 16
    trainX, trainY1,trainY16 = create_dataset(scaled_Close, look_back,TargetPrice)
    scaled_VWAP = scaled_VWAP[20:]
    trainX = trainX.reshape((trainX.shape[0], trainX.shape[1]))
    T_1 = np.column_stack((trainX, scaled_VWAP))
    scaled_X = T_1.reshape((T_1.shape[0], 1, T_1.shape[1]))
    scaled_X = scaled_X[-1000000:]
    trainY1 = trainY1[-1000000:]
    trainY16 = trainY16[-1000000:]
    name = AssetList1[i]
    print(name)
    model1 = Sequential()
    model1.add(LSTM(4, input_shape=(1, look_back+1)))
    model1.add(Dense(1))
    model1.compile(loss='mean_squared_error', optimizer='adam')
    model1.fit(scaled_X, trainY1, epochs=10, batch_size=40, verbose=2)
    AssetListT1[name]=model1
    model16 = Sequential()
    model16.add(LSTM(4, input_shape=(1, look_back+1)))
    model16.add(Dense(1))
    model16.compile(loss='mean_squared_error', optimizer='adam')
    model16.fit(scaled_X, trainY16, epochs=10, batch_size=40, verbose=2)
    AssetListT16[name]=model16

bnc



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Epoch 1/10
25000/25000 - 36s - loss: 2.9627e-04
Epoch 2/10
25000/25000 - 32s - loss: 1.3291e-06
Epoch 3/10
25000/25000 - 33s - loss: 1.2555e-06
Epoch 4/10
25000/25000 - 34s - loss: 1.2344e-06
Epoch 5/10
25000/25000 - 31s - loss: 1.2015e-06
Epoch 6/10
25000/25000 - 32s - loss: 1.1930e-06
Epoch 7/10
25000/25000 - 31s - loss: 1.1646e-06
Epoch 8/10
25000/25000 - 31s - loss: 1.1460e-06
Epoch 9/10
25000/25000 - 32s - loss: 1.1376e-06
Epoch 10/10
25000/25000 - 32s - loss: 1.1450e-06
Epoch 1/10
25000/25000 - 34s - loss: 8.5687e-05
Epoch 2/10
25000/25000 - 32s - loss: 5.3952e-07
Epoch 3/10
25000/25000 - 32s - loss: 3.4876e-07
Epoch 4/10
25000/25000 - 33s - loss: 3.2034e-07
Epoch 5/10
25000/25000 - 36s - loss: 2.9493e-07
Epoch 6/10
25000/25000 - 34s - loss: 2.9286e-07
Epoch 7/10
25000/25000 - 34s - loss: 2.8617e-07
Epoch 8/10
25000/25000 - 35s - loss: 2.7258e-07
Epoch 9/10
25000/25000 - 36s - loss: 2.5729e-07
Epoch 10/10
25000/25000 - 37s - loss: 2.5518e-07
btc
Epoch 1/10
25000/25000 - 40s - los

In [64]:
def calculate(z_test):
    global train_df
    result = pd.DataFrame(columns = ['timestamp','Asset_ID','Close'])
    for row in z_test.iterrows():
        timestamp = row[1]['timestamp']
        Asset_ID = row[1]['Asset_ID']
        Asset_ID = int(Asset_ID)
        Close = row[1]['Close']
        VWAP = row[1]['VWAP']
        listOfDataframe[Asset_ID] = listOfDataframe[Asset_ID].append(row[1])
        listOfDataframe[Asset_ID] = listOfDataframe[Asset_ID].drop_duplicates()
        dataset = listOfDataframe[Asset_ID]
        Prices = dataset[(dataset.timestamp<=timestamp)]['Close'][-4:]
        Prices = Prices.values
        X = np.append(Prices, VWAP)
        X = np.reshape(X,(-1,1))
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_Close = scaler.fit_transform(X)
        scaled_Close = scaled_Close.reshape((scaled_Close.shape[1], 1, scaled_Close.shape[0]))
        name =  AssetList1[Asset_ID]
        model1 = AssetListT1[name]
        yhat1 = model1.predict(scaled_Close)
        yhat1 = scaler.inverse_transform(yhat1)
        model16 = AssetListT16[name]
        yhat16 = model16.predict(scaled_Close)
        yhat16 = scaler.inverse_transform(yhat16)
        aa = {'timestamp':timestamp,'Asset_ID':Asset_ID,'Close':Close}
        train_df = train_df.append(aa,ignore_index=True)
        for i in range(1,17):
            bb = {'timestamp':timestamp+60*i,'Asset_ID':Asset_ID,'Close':yhat1*((15-i+1)/15)+yhat16*((i-1)/15)}
            result= result.append(bb,ignore_index=True)
    timestamp = z_test['timestamp'].values[0]
    x = result
    
    train_df=train_df.sort_values(by= 'timestamp',ascending=True)
    t = train_df[train_df.timestamp<=timestamp][-6000:]
    t = t.append(x,ignore_index=True)
    t[['timestamp', 'Asset_ID']] = t[['timestamp', 'Asset_ID']].astype(int)
    t.Close = t.Close.astype(float)
    price_column = 'Close'
    t['Time'] = pd.to_datetime(t['timestamp'], unit='s')
    t = t.drop_duplicates(subset = ['timestamp','Asset_ID'])
    k = calculate_target(t, details, price_column)
    target = k[k.index==timestamp]

    return target

In [ ]:
'''
timestamp,asset = 1632182400,11
Close1,Close16 = 234,236
'''

In [77]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [78]:
ids = list(details.Asset_ID)

In [31]:
'''
exam = supplement['timestamp'].unique().tolist()
exam = exam[:100]
pp =  pd.DataFrame()
for m in exam:
    z_test = supplement[supplement.timestamp==m]
    Target = calculate(z_test)
    Target = Target.values
    outcome = pd.DataFrame()
    order = z_test['Asset_ID'].values
    for k in order:
        line = ids.index(k)
        outcome=outcome.append({'Target':Target[0][line]},ignore_index=True)
    pp = pp.append(outcome,ignore_index=True)
    print(outcome)
'''

In [71]:
'''
answer = pd.read_csv("../input/g-research-crypto-forecasting/supplemental_train.csv")
answer = answer[answer.timestamp<=1632188400]['Target']
np.corrcoef(pp['Target'].values, answer.values)[0,1]
'''

In [79]:
for (test_df, sample_prediction_df) in iter_test:
    Target = calculate(test_df)
    Target = Target.values
    outcome = pd.DataFrame()
    order = test_df['Asset_ID'].values
    for k in order:
        line = ids.index(k)
        outcome=outcome.append({'Target':Target[0][line]},ignore_index=True)
    sample_prediction_df['Target'] = outcome['Target']
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
